***
# CHAPTER 9. DML
(Data Manipulation Language)  

### KEYWORD
* INSERT/ UPDATE/ DELETE /MERGE
* 다중테이블 (INSERT ALL, INSERT FIRST)

* AS OF TIMESTAMP (Flashback Query)
* ROLLBACK, COMMIT, SAVEPOINT

***

### before

* SELECT * FROM user_sys_privs;  #시스템권한 : 오라클db에 영향을 주는 권한
* SELECT * FROM user_tab_privs;  #오브젝트권한 : 테이블 소유자로부터 받은 오브젝트 권한(엑세스 가능)
* 테이블을 생성하려면 권한을 부여받아야 한다.
      1.dba로부터 직접 권한을 부여받을 수 있고,
      2.공통적인 권한들을 role이라는 곳에 모아놓고 그 role을 통해 권한을 받는다
* SELECT * FROM role_sys_privs; #CREATE TABLE 확인
* SELECT * FROM SESSION_PRIVS; #user_sys_privs와 role_sys_privs의 UNION
***

In [1]:
import cx_Oracle
import pandas as pd
conn = cx_Oracle.connect('hr/hr@localhost/xe')
c = conn.cursor()

## 9-1. INSERT, UPDATE, DELETE
### [ 연습문제 ]
#### (1) departments 테이블의 department_id, department_name, manager_id 데이터를 dept테이블로 insert 후, 영구저장하세요

In [3]:
# dept 테이블 생성
c.execute("CREATE TABLE dept(dept_id NUMBER(20), dept_name VARCHAR2(30), mgr NUMBER(20))")

# 데이터 삽입
c.execute("""INSERT INTO dept(dept_id, dept_name, mgr)
                    SELECT department_id, department_name, manager_id
                    FROM departments""")

# 출력
data = pd.read_sql("SELECT * FROM dept", conn)
print(data)

# 영구저장
conn.commit()

    DEPT_ID             DEPT_NAME    MGR
0        10        Administration  200.0
1        20             Marketing  201.0
2        30            Purchasing  114.0
3        40       Human Resources  203.0
4        50              Shipping  121.0
5        60                    IT  103.0
6        70      Public Relations  204.0
7        80                 Sales  145.0
8        90             Executive  100.0
9       100               Finance  108.0
10      110            Accounting  205.0
11      120              Treasury    NaN
12      130         Corporate Tax    NaN
13      140    Control And Credit    NaN
14      150  Shareholder Services    NaN
15      160              Benefits    NaN
16      170         Manufacturing    NaN
17      180          Construction    NaN
18      190           Contracting    NaN
19      200            Operations    NaN
20      210            IT Support    NaN
21      220                   NOC    NaN
22      230           IT Helpdesk    NaN
23      240     

#### (2) employees 테이블의 employee_id, last_name, first_name, hire_date, salary데이터를 emp 테이블에 insert를 하되 last_name과 first_name은 붙여서 입력하고 영구히 저장하세요.

In [5]:
# emp 테이블 생성
c.execute("CREATE TABLE emp(id NUMBER(20), name VARCHAR2(30), hire_date date, sal NUMBER(20), dept_id NUMBER(20))")

# 데이터 삽입
c.execute("""INSERT INTO emp(ID, NAME, hire_date, sal, dept_id)
                  SELECT employee_id, last_name||first_name, hire_date, salary, department_id
                  FROM employees""")
# 출력
data = pd.read_sql("SELECT * FROM emp", conn)
print(data)

# 영구저장
conn.commit()

      ID              NAME  HIRE_DATE    SAL  DEPT_ID
0    100        KingSteven 2003-06-17  24000     90.0
1    101      KochharNeena 2005-09-21  17000     90.0
2    102        De HaanLex 2001-01-13  17000     90.0
3    103   HunoldAlexander 2006-01-03   9000     60.0
4    104        ErnstBruce 2007-05-21   6000     60.0
5    105       AustinDavid 2005-06-25   4800     60.0
6    106    PataballaValli 2006-02-05   4800     60.0
7    107      LorentzDiana 2007-02-07   4200     60.0
8    108    GreenbergNancy 2002-08-17  12008    100.0
9    109      FavietDaniel 2002-08-16   9000    100.0
10   110          ChenJohn 2005-09-28   8200    100.0
11   111     SciarraIsmael 2005-09-30   7700    100.0
12   112  UrmanJose Manuel 2006-03-07   7800    100.0
13   113          PoppLuis 2007-12-07   6900    100.0
14   114       RaphaelyDen 2002-12-07  11000     30.0
15   115     KhooAlexander 2003-05-18   3100     30.0
16   116       BaidaShelli 2005-12-24   2900     30.0
17   117       TobiasSigal 2

#### (3) dept테이블에서 소속사원이 없는 부서 정보는 삭제하고 영구히 저장하세요.(CORRELATED SUBQUERY)

In [6]:
# 데이터 삭제
c.execute("""DELETE
             FROM dept d
             WHERE NOT EXISTS (SELECT 'x'
                               FROM emp
                               WHERE dept_id = d.dept_id)""")

# 출력
data = pd.read_sql("SELECT * FROM dept", conn)
print(data)

# 영구저장
conn.commit()

    DEPT_ID         DEPT_NAME  MGR
0        10    Administration  200
1        20         Marketing  201
2        30        Purchasing  114
3        40   Human Resources  203
4        50          Shipping  121
5        60                IT  103
6        70  Public Relations  204
7        80             Sales  145
8        90         Executive  100
9       100           Finance  108
10      110        Accounting  205


#### (4) emp 테이블에 dept_name varchar2(50) 컬럼을 추가하세요.

In [7]:
# 테이블 변경 (컬럼추가)
c.execute("ALTER TABLE emp ADD dept_name VARCHAR2(50)")

# 출력
data = pd.read_sql("SELECT * FROM emp", conn)
print(data)

      ID              NAME  HIRE_DATE    SAL  DEPT_ID DEPT_NAME
0    100        KingSteven 2003-06-17  24000     90.0      None
1    101      KochharNeena 2005-09-21  17000     90.0      None
2    102        De HaanLex 2001-01-13  17000     90.0      None
3    103   HunoldAlexander 2006-01-03   9000     60.0      None
4    104        ErnstBruce 2007-05-21   6000     60.0      None
5    105       AustinDavid 2005-06-25   4800     60.0      None
6    106    PataballaValli 2006-02-05   4800     60.0      None
7    107      LorentzDiana 2007-02-07   4200     60.0      None
8    108    GreenbergNancy 2002-08-17  12008    100.0      None
9    109      FavietDaniel 2002-08-16   9000    100.0      None
10   110          ChenJohn 2005-09-28   8200    100.0      None
11   111     SciarraIsmael 2005-09-30   7700    100.0      None
12   112  UrmanJose Manuel 2006-03-07   7800    100.0      None
13   113          PoppLuis 2007-12-07   6900    100.0      None
14   114       RaphaelyDen 2002-12-07  1

#### (5) emp 테이블의 dept_name 데이터는 dept테이블에 있는 dept_name 데이터를 기준으로 수정 후 영구저장하세요(CORRELATED SUBQUERY)

In [8]:
# 데이터 수정
c.execute("""UPDATE emp e  
            SET dept_name = (SELECT dept_name
                             FROM dept
                             WHERE dept_id = e.dept_id)""")
# 출력 
data = pd.read_sql("SELECT * FROM emp", conn)
print(data)

# 영구저장
conn.commit()

      ID              NAME  HIRE_DATE    SAL  DEPT_ID         DEPT_NAME
0    100        KingSteven 2003-06-17  24000     90.0         Executive
1    101      KochharNeena 2005-09-21  17000     90.0         Executive
2    102        De HaanLex 2001-01-13  17000     90.0         Executive
3    103   HunoldAlexander 2006-01-03   9000     60.0                IT
4    104        ErnstBruce 2007-05-21   6000     60.0                IT
5    105       AustinDavid 2005-06-25   4800     60.0                IT
6    106    PataballaValli 2006-02-05   4800     60.0                IT
7    107      LorentzDiana 2007-02-07   4200     60.0                IT
8    108    GreenbergNancy 2002-08-17  12008    100.0           Finance
9    109      FavietDaniel 2002-08-16   9000    100.0           Finance
10   110          ChenJohn 2005-09-28   8200    100.0           Finance
11   111     SciarraIsmael 2005-09-30   7700    100.0           Finance
12   112  UrmanJose Manuel 2006-03-07   7800    100.0           

## 9-2. INSERT ALL, INSERT FIRST 
다중테이블

### < UNCONDITIONAL INSERT ALL >

In [11]:
# 테이블생성
c.execute("""CREATE TABLE sal_history
          AS
          SELECT employee_id, hire_date, salary
          FROM employees
          WHERE 1=2""") #테이블 구조만 복제 
    
c.execute("""CREATE TABLE mgr_history
          AS
          SELECT employee_id, manager_id, salary
          FROM employees
          WHERE 1=2""")   

# 두 개의 테이블로 데이터 동시 삽입

c.execute("""INSERT ALL
          INTO sal_history(employee_id, hire_date, salary) 
          VALUES(EMPID,HIREDATE,SAL)
          
          INTO mgr_history(employee_id, manager_id, salary) 
          VALUES(EMPID,MGR,SAL)
          
          SELECT employee_id EMPID, hire_date HIREDATE,salary SAL, manager_id MGR
          FROM employees
          WHERE employee_id > 200""")

# 출력
sal = pd.read_sql("SELECT * FROM sal_history", conn)
mgr = pd.read_sql("SELECT * FROM mgr_history", conn)

print(sal)
print(' ')
print(mgr)

# 작업수행 취소
conn.rollback()

   EMPLOYEE_ID  HIRE_DATE   SALARY
0          201 2004-02-17  13000.0
1          202 2005-08-17   6000.0
2          203 2002-06-07   6500.0
3          204 2002-06-07  10000.0
4          205 2002-06-07  12008.0
5          206 2002-06-07   8300.0
 
   EMPLOYEE_ID  MANAGER_ID   SALARY
0          201         100  13000.0
1          202         201   6000.0
2          203         101   6500.0
3          204         101  10000.0
4          205         101  12008.0
5          206         205   8300.0


### < CONDITIONAL INSERT ALL >

In [12]:
# 테이블생성
c.execute("""CREATE TABLE emp_history
          AS
          SELECT employee_id, hire_date, salary
          FROM employees
          WHERE 1=2""")   #테이블 구조만 복제 

c.execute("""CREATE TABLE emp_sales
          AS
          SELECT employee_id, commission_pct, salary
          FROM employees
          WHERE 1=2""")  

# 조건부 데이터 삽입
c.execute("""INSERT ALL
          WHEN HIREDATE < to_date('2005-01-01','YYYY-MM-DD') THEN
          INTO emp_history(employee_id, hire_date, salary) 
          VALUES(EMPID,HIREDATE,SAL)
          
          WHEN COMM IS NOT NULL THEN\
          INTO emp_sales(employee_id, commission_pct, salary) 
          VALUES(EMPID,COMM,SAL)
          
          SELECT employee_id EMPID, hire_date HIREDATE,salary SAL, commission_pct COMM
          FROM employees""")
# 출력
history = pd.read_sql("SELECT * FROM emp_history", conn)
sales = pd.read_sql("SELECT * FROM emp_sales", conn)

print(history)
print(' ')
print(sales)

# 작업수행 취소 
conn.rollback()

    EMPLOYEE_ID  HIRE_DATE   SALARY
0           100 2003-06-17  24000.0
1           102 2001-01-13  17000.0
2           108 2002-08-17  12008.0
3           109 2002-08-16   9000.0
4           114 2002-12-07  11000.0
5           115 2003-05-18   3100.0
6           120 2004-07-18   8000.0
7           122 2003-05-01   7900.0
8           133 2004-06-14   3300.0
9           137 2003-07-14   3600.0
10          141 2003-10-17   3500.0
11          145 2004-10-01  14000.0
12          156 2004-01-30  10000.0
13          157 2004-03-04   9500.0
14          158 2004-08-01   9000.0
15          174 2004-05-11  11000.0
16          184 2004-01-27   4200.0
17          192 2004-02-04   4000.0
18          200 2003-09-17   4400.0
19          201 2004-02-17  13000.0
20          203 2002-06-07   6500.0
21          204 2002-06-07  10000.0
22          205 2002-06-07  12008.0
23          206 2002-06-07   8300.0
 
    EMPLOYEE_ID  COMMISSION_PCT   SALARY
0           145            0.40  14000.0
1           146 

### < CONDITIONAL INSERT FIRST >

In [13]:
# 테이블생성
c.execute("""CREATE TABLE sal_low
          AS
          SELECT employee_id, last_name, salary
          FROM employees
          WHERE 1=2""")  #테이블의 구조만 복제 

c.execute("""CREATE TABLE sal_mid
          AS
          SELECT employee_id, last_name, salary
          FROM employees
          WHERE 1=2""")

c.execute("""CREATE TABLE sal_high
          AS
          SELECT employee_id, last_name, salary
          FROM employees
          WHERE 1=2""")

# 조건부 데이터 삽입(조건에 만족한 첫번째만 작업수행)
c.execute("""INSERT FIRST
          WHEN salary < 5000 THEN
          INTO sal_low(employee_id, last_name, salary) 
          VALUES (employee_id, last_name, salary)
          
          WHEN salary between 5000 and 10000 THEN
          INTO sal_mid(employee_id, last_name, salary) 
          VALUES (employee_id, last_name, salary)
          
          ELSE
          INTO sal_high(employee_id, last_name, salary) 
          VALUES (employee_id, last_name, salary)
          
          SELECT employee_id, last_name, salary
          FROM employees""")

# 출력
low = pd.read_sql("SELECT * FROM sal_low", conn)
mid = pd.read_sql("SELECT * FROM sal_mid", conn)
high = pd.read_sql("SELECT * FROM sal_high", conn)

print('---------------LOW---------------')
print(low)
print('---------------MID---------------')
print(mid)
print('---------------HIGH---------------')
print(high)

# 작업수행 취소 
conn.rollback()

---------------LOW---------------
    EMPLOYEE_ID    LAST_NAME  SALARY
0           105       Austin  4800.0
1           106    Pataballa  4800.0
2           107      Lorentz  4200.0
3           115         Khoo  3100.0
4           116        Baida  2900.0
5           117       Tobias  2800.0
6           118       Himuro  2600.0
7           119   Colmenares  2500.0
8           125        Nayer  3200.0
9           126  Mikkilineni  2700.0
10          127       Landry  2400.0
11          128       Markle  2200.0
12          129       Bissot  3300.0
13          130     Atkinson  2800.0
14          131       Marlow  2500.0
15          132        Olson  2100.0
16          133       Mallin  3300.0
17          134       Rogers  2900.0
18          135          Gee  2400.0
19          136   Philtanker  2200.0
20          137       Ladwig  3600.0
21          138       Stiles  3200.0
22          139          Seo  2700.0
23          140        Patel  2500.0
24          141         Rajs  3500.0
25  

## 9-3. MERGE
조건에 따라서 INSERT, UPDATE, DELETE 작업을 한 번에 할 수 있다.

In [14]:
# 테이블생성 
c.execute("""CREATE TABLE oltp_emp
          AS 
          SELECT employee_id, last_name, salary, department_id
          FROM employees""")
   

c.execute("""CREATE TABLE dw_emp
          AS 
          SELECT employee_id, last_name, salary, department_id
          FROM employees
          WHERE department_id = 20""")

# dw_emp 테이블 출력
dw = pd.read_sql("SELECT * FROM dw_emp", conn)
print('----------------------dw_emp----------------------')
print(dw)

----------------------dw_emp----------------------
   EMPLOYEE_ID  LAST_NAME   SALARY  DEPARTMENT_ID
0          201  Hartstein  13000.0             20
1          202        Fay   6000.0             20


In [15]:
# 테이블에 컬럼추가
c.execute("ALTER TABLE oltp_emp ADD flag CHAR(1)")

# oltp_emp 테이블 갱신
c.execute("""UPDATE oltp_emp
          SET flag = 'd'
          WHERE employee_id = 202""")

c.execute("""UPDATE oltp_emp
          SET salary  = 20000
          WHERE employee_id = 201""")

# 저장
conn.commit()

# 테이블 출력
oltp = pd.read_sql("SELECT * FROM oltp_emp WHERE employee_id = 202", conn)
dw = pd.read_sql("SELECT * FROM dw_emp", conn)

print('----------------------oltp_emp----------------------')
print(oltp)
print('----------------------dw_emp----------------------')
print(dw)

----------------------oltp_emp----------------------
   EMPLOYEE_ID LAST_NAME  SALARY  DEPARTMENT_ID FLAG
0          202       Fay  6000.0             20    d
----------------------dw_emp----------------------
   EMPLOYEE_ID  LAST_NAME   SALARY  DEPARTMENT_ID
0          201  Hartstein  13000.0             20
1          202        Fay   6000.0             20


In [16]:
# 데이터 이행작업
c.execute("""MERGE INTO dw_emp c 
              USING (SELECT * FROM oltp_emp ) e      
              ON (c.employee_id = e.employee_id)
              
              WHEN MATCHED THEN
                  UPDATE SET
                      c.last_name     = e.last_name,
                      c.salary        = e.salary * 1.1,
                      c.department_id = e.department_id
                  DELETE WHERE 
                      e.flag = 'd'
                  
             WHEN NOT MATCHED THEN
                 INSERT(c.employee_id, c.last_name, c.salary, c.department_id)
                 VALUES(e.employee_id, e.last_name, e.salary, e.department_id)""")
             
             
             
cn = pd.read_sql("SELECT count(*) FROM dw_emp", conn)
dw_emp = pd.read_sql("SELECT * FROM dw_emp WHERE employee_id = 202", conn)

print(cn)
print('----------------------dw_emp----------------------')
print(dw_emp)

   COUNT(*)
0       106
----------------------dw_emp----------------------
Empty DataFrame
Columns: [EMPLOYEE_ID, LAST_NAME, SALARY, DEPARTMENT_ID]
Index: []


In [17]:
conn.rollback()

dw_emp = pd.read_sql("SELECT * FROM dw_emp WHERE employee_id = 202", conn)
print(dw_emp)

   EMPLOYEE_ID LAST_NAME  SALARY  DEPARTMENT_ID
0          202       Fay  6000.0             20


## 9-4. Flashback Query / Savepoint
### < Flashback Query _ AS OF TIMESTAMP >

In [19]:
# 테이블 생성 및 조회
c.execute("""CREATE TABLE hr.emp_30
          AS SELECT * FROM hr.employees WHERE department_id = 30""")

data = pd.read_sql("""SELECT employee_id, salary, department_id 
                   FROM hr.emp_30""", conn)
print(data)

# 현재시간
data = pd.read_sql("SELECT SYSTIMESTAMP FROM dual", conn)
print(' ')
print(data)

   EMPLOYEE_ID   SALARY  DEPARTMENT_ID
0          114  11000.0             30
1          115   3100.0             30
2          116   2900.0             30
3          117   2800.0             30
4          118   2600.0             30
5          119   2500.0             30
 
             SYSTIMESTAMP
0 2017-11-13 13:01:54.073


In [20]:
# 데이터 갱신  
c.execute("""UPDATE hr.emp_30
          SET salary = 30000
          WHERE employee_id = 114""")

conn.commit()

# commit 후 갱신된 데이터 조회
data = pd.read_sql("""SELECT employee_id, salary 
                   FROM hr.emp_30 
                   WHERE employee_id = 114""", conn)
print(data)

   EMPLOYEE_ID   SALARY
0          114  30000.0


#### commit 이전의 데이터 조회
commit을 하더라도 갱신 전 데이터는 undo에 저장되어있다.
집계값을 구하는 동안 DML수정 못하도록(읽기의 일관성) 이전값을 undo가 가지고 있는 것

In [24]:
# 1분전 내용을 출력
data = pd.read_sql("""SELECT employee_id, salary
                   FROM hr.emp_30 
                   AS OF TIMESTAMP (SYSTIMESTAMP - INTERVAL '1' MINUTE)
                   WHERE employee_id = 114""", conn)
print(data)

   EMPLOYEE_ID   SALARY
0          114  11000.0


### < SAVEPOINT >

In [73]:
# 테이블생성
c.execute("CREATE TABLE TEST(ID NUMBER)")

# 데이터 삽입
c.execute("INSERT INTO TEST(ID) VALUES(1)")

# 저장되기 이전에 되돌아올 포인트 지정 
c.execute("SAVEPOINT A") 
data = pd.read_sql("SELECT * FROM TEST", conn)
print(data)

   ID
0   1


In [74]:
c.execute("INSERT INTO TEST(ID) VALUES(2)")

c.execute("SAVEPOINT b")

data = pd.read_sql("SELECT * FROM TEST", conn)
print(data)

   ID
0   1
1   2


In [75]:
c.execute("INSERT INTO TEST(ID) VALUES(3)")

data = pd.read_sql("SELECT * FROM TEST", conn)
print(data)

   ID
0   1
1   2
2   3


In [76]:
# A 지점으로 되돌아가기
c.execute("ROLLBACK TO A")  

data = pd.read_sql("SELECT * FROM TEST", conn)
print(data)

   ID
0   1


In [77]:
# conn.rollback()   마지막 commit된 상태로 되돌아가기
# conn.commit()

In [ ]:
c.close()
conn.close()